In [5]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
file_path = '/mnt/d/codes/Datasets/datasetNew.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (last row)
constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values
df = df.iloc[:-1]

# Extract items and groups
items = []
groups = df['Group'].unique()
num_resources = len(constraint_row)

# Create a list of items with their resource usage and values
for _, row in df.iterrows():
    item = {
        'Group': row['Group'],
        'Item': row['Item'],
        'Value': row['Value'],
        'Resources': row[3:].values.astype(int)
    }
    items.append(item)

# Function to generate an initial solution
def initial_solution(items):
    solution = []
    groups = set(item['Group'] for item in items)
    for group in groups:
        group_items = [item for item in items if item['Group'] == group]
        solution.append(np.random.choice(group_items))
    return solution

# Function to evaluate the solution
def evaluate_solution(solution, constraints):
    group_values = {group: 0 for group in groups}
    total_resources = np.sum([item['Resources'] for item in solution], axis=0)
    valid = np.all(total_resources <= constraints)
    
    for item in solution:
        group_values[item['Group']] += item['Value']
    
    if not valid:
        return {group: -1 for group in groups}
    
    return group_values

# Function to generate a neighboring solution
def neighbor_solution(solution, items):
    new_solution = solution.copy()
    groups = list(set(item['Group'] for item in solution))
    group = np.random.choice(groups)
    group_items = [item for item in items if item['Group'] == group]
    
    if len(group_items) > 1:
        new_item = np.random.choice(group_items)
        idx = next(i for i, item in enumerate(new_solution) if item['Group'] == group)
        new_solution[idx] = new_item

    return new_solution

# Simulated Annealing algorithm implementation
def simulated_annealing(items, constraints, initial_temp=1000, cooling_rate=0.003, max_iterations=1000):
    current_solution = initial_solution(items)
    best_solution = current_solution
    current_values = evaluate_solution(current_solution, constraints)
    best_values = current_values
    temp = initial_temp
    
    for _ in range(max_iterations):
        new_solution = neighbor_solution(current_solution, items)
        new_values = evaluate_solution(new_solution, constraints)
        
        for group in best_values:
            if new_values[group] > best_values[group]:
                current_solution = new_solution
                current_values = new_values
                best_solution = new_solution
                best_values = new_values
        
        temp *= (1 - cooling_rate)
    
    return best_solution, best_values

# Run Simulated Annealing
best_solution, best_values = simulated_annealing(items, constraint_row)

# Find the overall best value
overall_best_value = max(best_values.values())

# Output the best value
print(f"Best value: {overall_best_value}")


/tmp/ipykernel_23076/2252354051.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values


Best value: 199.0
